# Аналитика злоупотреблений в программе лояльности:

В сервисе «денежные переводы» запустили акцию «пригласи друга»: можно поделиться
промокодом. «Промокод» дает возможность отправить перевод без комиссии одному
пользователю 1 раз. За каждого нового пользователя приглашающий получает «купон»
на отправку перевод без комиссии для себя.
Поступил сигнал, что появились люди, которые злоупотребляют условиями акции.
Известны два промокода, который замешены в подобном: FYFWV3 и AG5IR4. Во
вложении выгрузка с операциями за какой-то период
Задача:

 проанализировать данные и разметить какие операции плохие, а какие
нормальные (плохие = клиенты злоупотребили условиями сервиса)

 сформировать резюме по итогам анализа: кратко и понятно предоставить
результат анализа (подтвердился ли сигнал, насколько проблема массовая и т.д.)

 предложить способ как быстро выявлять подобную активность

 предложить варианты реагирования

Доп информация:

o   По условиям акции максимальное количество использований промокода – 100. На
каждом промокоде существует счетчик, который фиксирует текущее количество
промокодов

o   Уникального пользователя в онлайне определяет только телефон

In [6]:
import pandas as pd
import numpy as np

In [7]:
cft = pd.read_excel('cft.xlsx')

In [8]:
cft.head()
#Посмотрим на хэд, просто визуально оценим данные

,ID операции,Дата отправки,Статус платежа,Сумма,Валюта,Дата выдачи,Направление,ФИО отправителя после преобразования,Телефон отправителя_ХЭШ,ФИО получателя после преобразования,...,ФИО реального получателя после преобразования,Телефон реального получателя_ХЭШ,Агент отправки,Город выдачи,Пункт выдачи,Признак скидки,Идентификатор промокода,id юзера-приглашателя,ФИО приглашателя после преобразования,Телефон (приглашатель)_ХЭШ
0,2707850631,2018-08-18,Выдан,1000,Российский рубль,2018-08-18,ТАДЖИКИСТАН,АРИКОВ ДАЙФОРУР,\=2$Xh\b}gAk,ЗАНАРУРОВА БУРНОФА ЗОРИДОВНА,...,ЗАНАРУРОВА БУРНОФА ЗОРИДОВНА,\?2&[aa_td=hZ,"""Агент для online переводов""",ДУШАНБЕ,"ST00985 ЗАО ""Казкоммерцбанк Таджикистан""",По промокоду,AIPHXT,348714,РАКАЧФОРА КАНУЧЕШЧИ ШУЧРАЗ,\=2-[ga_wg8m
1,2708069031,2018-08-18,Выдан,200,Доллар США,2018-08-21,УЗБЕКИСТАН,ЗИРОВОВ ДЕРФОР НАЧФУРРАЕВИЧ,\=2*Ui[`}i9i,PUQMAPOV QAMAT QO'CFQOQOVICF,...,PUQMAPOV QAMAT QO'CFQOQOVICF,"\?2,[e[gz`>hR","""Агент для online переводов""",ПАЙАРЫК,"AG60375 ОАКБ ""Агробанк"", Пайарикский филиал",По промокоду,MPUYXI,227215,ЗИРОВОВ ДЕРФОР НАЧФУРРАЕВИЧ,\=2*Tccbza@g
2,2708071431,2018-08-18,Выдан,200,Доллар США,2018-08-22,УЗБЕКИСТАН,ШОЗАКОВ КАНЗУЧ,\=2$Xe[ezeAi,ЗАНОЕВА НОРИЧА ШАЧИШОВНА,...,ЗАНОЕВА НОРИЧА ШАЧИШОВНА,"\?2,[e]axeAh[","""Агент для online переводов""",ШОФИРКАН,"AH20128 Народный банк, Филиал Шофиркан",По промокоду,LXVH52,56088,ШОЗАКОВ КАНЗУЧ,\=2)Tfbc}`@l
3,2708306031,2018-08-19,Выдан,300,Доллар США,2018-08-21,УЗБЕКИСТАН,ЗАРУРРОЕВ АФИФДЕР ЧУЗЗАКОВИЧ,\=2-[g]`ya=h,КУЗАЕВ РИРКУЧОР АЗРАКОВИЧ,...,КУЗАЕВ РИРКУЧОР АЗРАКОВИЧ,"\?2,[hc`|d>hX","""Агент для online переводов""",САМАРКАНД,"AG60374 ОАКБ ""Агробанк"", Сиабский филиал",По промокоду,YEM8TE,431901,ЗАРУРРОЕВ АФИФДЕР ЧУЗЗАКОВИЧ,\=2*Vj[auf=k
4,2709021031,2018-08-20,Выдан,250,Доллар США,2018-08-22,УЗБЕКИСТАН,ЗОЙЧИДОЕВ ДАШЧОКЖОН ШАКЧОДОЕВИЧ,\=2&Xhcc}e8i,АДРУРАШАЗОВ БУРОК НОЧДОЙЕВИЧ,...,АДРУРАШАЗОВ БУРОК НОЧДОЙЕВИЧ,"\?2,RaZ_t`8dR%rB","""Агент для online переводов""",ДЖАМБАЙ,"AH20050 Народный банк, Филиал Жамбай",По купону,NaN,0,,NaN


In [9]:
cft.shape
#3306, 21 столбец

(3306, 21)

In [10]:
cft.nunique()
#проверим кол-во уникальных значений в каждой строке
#уникальный только id оперции, остальные значения могут повторятся или чатсь строк в колонках пустая

ID операции                                      3306
Дата отправки                                      31
Статус платежа                                      4
Сумма                                             353
Валюта                                              7
Дата выдачи                                        32
Направление                                        18
ФИО отправителя после преобразования             1784
Телефон отправителя_ХЭШ                          2576
ФИО получателя после преобразования              2499
Телефон получателя_ХЭШ                           2379
ФИО реального получателя после преобразования    2396
Телефон реального получателя_ХЭШ                 2224
Агент отправки                                      1
Город выдачи                                      327
Пункт выдачи                                     1335
Признак скидки                                      2
Идентификатор промокода                           850
id юзера-приглашателя       

In [11]:
cft.dtypes
#Проверим тип данных

ID операции                                               int64
Дата отправки                                    datetime64[ns]
Статус платежа                                           object
Сумма                                                     int64
Валюта                                                   object
Дата выдачи                                      datetime64[ns]
Направление                                              object
ФИО отправителя после преобразования                     object
Телефон отправителя_ХЭШ                                  object
ФИО получателя после преобразования                      object
Телефон получателя_ХЭШ                                   object
ФИО реального получателя после преобразования            object
Телефон реального получателя_ХЭШ                         object
Агент отправки                                           object
Город выдачи                                             object
Пункт выдачи                            

In [12]:
cft["Дата отправки"].min()
#18 августа 2018 года

Timestamp('2018-08-18 00:00:00')

In [13]:
cft["Дата отправки"].max()
#17 сентября 2018 года

Timestamp('2018-09-17 00:00:00')

In [14]:
#выгрузка ровзно за 1 месяц: с 18.08.2018 по 17.09.2018

In [15]:
cft.isna().sum()
#посмотрим на столбцы,где присутствуют пустые значения:

ID операции                                         0
Дата отправки                                       0
Статус платежа                                      0
Сумма                                               0
Валюта                                              0
Дата выдачи                                       137
Направление                                         0
ФИО отправителя после преобразования                0
Телефон отправителя_ХЭШ                             0
ФИО получателя после преобразования                 0
Телефон получателя_ХЭШ                              0
ФИО реального получателя после преобразования     151
Телефон реального получателя_ХЭШ                  286
Агент отправки                                      0
Город выдачи                                      151
Пункт выдачи                                        0
Признак скидки                                      0
Идентификатор промокода                          1323
id юзера-приглашателя       

Кратко:

0.не все платежи выданы 

1.сответственно, не везде есть реальный получатель и город выдачи (при этом пустых значений с ФИО получателя и городом на 14 больше, чем платежей без даты выдачи, получается минимум  14 платежей имеют фио и город выдачи,не не имеют дату выдачи)

In [16]:
cft.columns = cft.columns.str.lower().str.replace(" ", "_")
#для более удобной работы немного поменяем названия колонок: уберём верхний регистр и добавим _ м/у словами
#но оставим русский язык - мы же юзерфрендли

In [17]:
cft.head()

,id_операции,дата_отправки,статус_платежа,сумма,валюта,дата_выдачи,направление,фио_отправителя_после_преобразования,телефон_отправителя_хэш,фио_получателя_после_преобразования,...,фио_реального_получателя_после_преобразования,телефон_реального_получателя_хэш,агент_отправки,город_выдачи,пункт_выдачи,признак_скидки,идентификатор_промокода,id_юзера-приглашателя,фио_приглашателя_после_преобразования,телефон_(приглашатель)_хэш
0,2707850631,2018-08-18,Выдан,1000,Российский рубль,2018-08-18,ТАДЖИКИСТАН,АРИКОВ ДАЙФОРУР,\=2$Xh\b}gAk,ЗАНАРУРОВА БУРНОФА ЗОРИДОВНА,...,ЗАНАРУРОВА БУРНОФА ЗОРИДОВНА,\?2&[aa_td=hZ,"""Агент для online переводов""",ДУШАНБЕ,"ST00985 ЗАО ""Казкоммерцбанк Таджикистан""",По промокоду,AIPHXT,348714,РАКАЧФОРА КАНУЧЕШЧИ ШУЧРАЗ,\=2-[ga_wg8m
1,2708069031,2018-08-18,Выдан,200,Доллар США,2018-08-21,УЗБЕКИСТАН,ЗИРОВОВ ДЕРФОР НАЧФУРРАЕВИЧ,\=2*Ui[`}i9i,PUQMAPOV QAMAT QO'CFQOQOVICF,...,PUQMAPOV QAMAT QO'CFQOQOVICF,"\?2,[e[gz`>hR","""Агент для online переводов""",ПАЙАРЫК,"AG60375 ОАКБ ""Агробанк"", Пайарикский филиал",По промокоду,MPUYXI,227215,ЗИРОВОВ ДЕРФОР НАЧФУРРАЕВИЧ,\=2*Tccbza@g
2,2708071431,2018-08-18,Выдан,200,Доллар США,2018-08-22,УЗБЕКИСТАН,ШОЗАКОВ КАНЗУЧ,\=2$Xe[ezeAi,ЗАНОЕВА НОРИЧА ШАЧИШОВНА,...,ЗАНОЕВА НОРИЧА ШАЧИШОВНА,"\?2,[e]axeAh[","""Агент для online переводов""",ШОФИРКАН,"AH20128 Народный банк, Филиал Шофиркан",По промокоду,LXVH52,56088,ШОЗАКОВ КАНЗУЧ,\=2)Tfbc}`@l
3,2708306031,2018-08-19,Выдан,300,Доллар США,2018-08-21,УЗБЕКИСТАН,ЗАРУРРОЕВ АФИФДЕР ЧУЗЗАКОВИЧ,\=2-[g]`ya=h,КУЗАЕВ РИРКУЧОР АЗРАКОВИЧ,...,КУЗАЕВ РИРКУЧОР АЗРАКОВИЧ,"\?2,[hc`|d>hX","""Агент для online переводов""",САМАРКАНД,"AG60374 ОАКБ ""Агробанк"", Сиабский филиал",По промокоду,YEM8TE,431901,ЗАРУРРОЕВ АФИФДЕР ЧУЗЗАКОВИЧ,\=2*Vj[auf=k
4,2709021031,2018-08-20,Выдан,250,Доллар США,2018-08-22,УЗБЕКИСТАН,ЗОЙЧИДОЕВ ДАШЧОКЖОН ШАКЧОДОЕВИЧ,\=2&Xhcc}e8i,АДРУРАШАЗОВ БУРОК НОЧДОЙЕВИЧ,...,АДРУРАШАЗОВ БУРОК НОЧДОЙЕВИЧ,"\?2,RaZ_t`8dR%rB","""Агент для online переводов""",ДЖАМБАЙ,"AH20050 Народный банк, Филиал Жамбай",По купону,NaN,0,,NaN


In [18]:
cft['признак_скидки'].unique()

array(['По промокоду', 'По купону'], dtype=object)

In [19]:
cft.query("признак_скидки == 'По купону'").shape[0]
#транзакций по купону

1323

In [20]:
cft.query("признак_скидки == 'По промокоду'").shape[0]
#транзакций по промокоду

1983

промокоды, которыми злоупотребили: FYFWV3 и AG5IR4

In [21]:
evill = cft.query("идентификатор_промокода == 'FYFWV3' or идентификатор_промокода == 'AG5IR4'")

In [22]:
evill.shape[0]
#39 переводов по указанным промокодам

39

In [23]:
evill[['фио_приглашателя_после_преобразования', 'фио_отправителя_после_преобразования', 'id_операции']] \
    .groupby(['фио_приглашателя_после_преобразования', 'фио_отправителя_после_преобразования'], as_index = False) \
    .agg({"id_операции": "count"}).rename(columns={"id_операции": "кол-во"})

,фио_приглашателя_после_преобразования,фио_отправителя_после_преобразования,кол-во
0,ЧАШИКФОРА ШОШЧУШ ШАЧИШ,УКАЗОВ ЧАУШРЖОН РОРИЧРУРОВИЧ,1
1,ЧАШИКФОРА ШОШЧУШ ШАЧИШ,ЧАШИКФОРА ШОШЧУШ,2
2,ЧАШИКФОРА ШОШЧУШ ШАЧИШ,ЧАШИКФОРА ШОШЧУШ ШАЧИШ,18
3,ЧУЗЗАКОВ КИЧФОШИР КУКИНОВИЧ,АДРУРРАЕВ АДРУШАРИК АДРУРШАКИЗОВИЧ,1
4,ЧУЗЗАКОВ КИЧФОШИР КУКИНОВИЧ,АЗРАЧОВ НОФИКЖОН ОРИКЖОНОВИЧ,1
5,ЧУЗЗАКОВ КИЧФОШИР КУКИНОВИЧ,ДОЙКУЧОРОВ ДАШЗИЁЧ ЗУРАШЕВИЧ,2
6,ЧУЗЗАКОВ КИЧФОШИР КУКИНОВИЧ,ЗИРИРОВ АДРУВОШИР ЖАКОРРИНОВИЧ,10
7,ЧУЗЗАКОВ КИЧФОШИР КУКИНОВИЧ,КИЧФАЕВ ЗУРАНДОЙ КУШАККАРИДЧОШИКОВИЧ,1
8,ЧУЗЗАКОВ КИЧФОШИР КУКИНОВИЧ,УКАЧОВ КАРЁЧДЕР ЗОИДЖОНОВИЧ,2
9,ЧУЗЗАКОВ КИЧФОШИР КУКИНОВИЧ,ЧУФИЕВ ЗУЧЗУНКАКАЗ ДАШЗИЕЧОВИЧ,1


ЧАШИКФОРА ШОШЧУШ ШАЧИШ	злоупотребил акцией 18 раз, ЧУЗЗАКОВ КИЧФОШИР КУКИНОВИЧ	- 9 раз. И это без учета вероятного использования неполного имени (фамилия и имя без отчества)

Проверим, были ли ещё промокоды, которыми также злоупотребляли:

In [24]:
cft.query("признак_скидки == 'По промокоду'") \
    .query("статус_платежа == 'Выдан'") \
    [['id_юзера-приглашателя', 'фио_приглашателя_после_преобразования', 'телефон_отправителя_хэш', 
      'фио_отправителя_после_преобразования', 'id_операции', 'идентификатор_промокода']] \
    .groupby(['id_юзера-приглашателя', 'фио_приглашателя_после_преобразования', 
              'фио_отправителя_после_преобразования', 'идентификатор_промокода'], as_index = False) \
    .agg({"id_операции": "count"}).sort_values(by='id_операции', ascending=False).query("id_операции > 1")

,id_юзера-приглашателя,фио_приглашателя_после_преобразования,фио_отправителя_после_преобразования,идентификатор_промокода,id_операции
254,130381,ШИРИППОВЗРИЙ ЗЕЧБЕЙ ШЧЕРРИЕВИЧ,ШИРИППОВЗРИЙ ЗЕЧБЕЙ ШЧЕРРИЕВИЧ,V65XXZ,19
1117,458182,ЧАШИКФОРА ШОШЧУШ ШАЧИШ,ЧАШИКФОРА ШОШЧУШ ШАЧИШ,FYFWV3,18
1022,416868,ДОЙДОДОЕВ РИРШОРЖОН КУЧАЗОВИЧ,ЮРРАШЕВ ИЗКАЗ,675LKD,17
497,233360,РЖУЧАЕВ РАЮКЖОН ЗОИЧШОНОВИЧ,АЗВАРОВ РАНОАЗШОШ ЧАШКАЗЖОНОВИЧ,CIUWW2,16
328,157136,ПАЯНОВ ЧУЗЗАК АДРУЧАЗУРОВИЧ,ПАЯНОВ ЧУЗЗАК,G46YBL,13
...,...,...,...,...,...
811,370613,ЗУРЗАНОВ КАЙЧАКДЕР ЗУРАЙКАНРУРОВИЧ,ЗУРЗАНОВ КАЙЧАКДЕР ЗУРАЙКАНРУРОВИЧ,SD35LE,2
162,81594,КАКАЗРАЧИКОВ ЧУЗЗАК ЧОДИКРУРОВИЧ,ЧАШИКОВ ШАЧЧУШ КУЬКИНЧОНОВИЧ,PEG9KL,2
692,333836,РИК РКИЗЧИЙ ИБОЧЬЕВИЧ,,3CD3WF,2
617,298126,АРИЕВ ДАШЗИЁЧ АДРУКАРИРОВИЧ,РЖАДОЧОВ ЧУЗЗАК ШАКИРУРРОЕВИЧ,BM119L,2


Очевидно, да. 198 промокодами воспользовались в одной паре ПРИГЛАШАТЕЛЬ-ОТПРАВИТЕЛЬ более одного раза. Для чистоты эксперимента возьмём только те, где ситуация повторялась чаще двух раз (сделаем погрешность на тёзок)

In [25]:
cft.query("признак_скидки == 'По промокоду'") \
    .query("статус_платежа == 'Выдан'") \
    [['id_юзера-приглашателя', 'фио_приглашателя_после_преобразования', 'телефон_отправителя_хэш', 
      'фио_отправителя_после_преобразования', 'id_операции', 'идентификатор_промокода']] \
    .groupby(['id_юзера-приглашателя', 'фио_приглашателя_после_преобразования', 
              'фио_отправителя_после_преобразования', 'идентификатор_промокода'], as_index = False) \
    .agg({"id_операции": "count"}).sort_values(by='id_операции', ascending=False).query("id_операции > 2")

,id_юзера-приглашателя,фио_приглашателя_после_преобразования,фио_отправителя_после_преобразования,идентификатор_промокода,id_операции
254,130381,ШИРИППОВЗРИЙ ЗЕЧБЕЙ ШЧЕРРИЕВИЧ,ШИРИППОВЗРИЙ ЗЕЧБЕЙ ШЧЕРРИЕВИЧ,V65XXZ,19
1117,458182,ЧАШИКФОРА ШОШЧУШ ШАЧИШ,ЧАШИКФОРА ШОШЧУШ ШАЧИШ,FYFWV3,18
1022,416868,ДОЙДОДОЕВ РИРШОРЖОН КУЧАЗОВИЧ,ЮРРАШЕВ ИЗКАЗ,675LKD,17
497,233360,РЖУЧАЕВ РАЮКЖОН ЗОИЧШОНОВИЧ,АЗВАРОВ РАНОАЗШОШ ЧАШКАЗЖОНОВИЧ,CIUWW2,16
328,157136,ПАЯНОВ ЧУЗЗАК АДРУЧАЗУРОВИЧ,ПАЯНОВ ЧУЗЗАК,G46YBL,13
...,...,...,...,...,...
50,19457,ПЧОЗАЗОВ ВИРЗОЧ АРЕРЗАНРЧОВИЧ,ЗУЧАЕВА ШАШЧИНИЗО ЗУЧЗУНОВНА,W5TY6Y,3
765,355338,АРИКОВ ЗАЙРОРИК ЕРУДОВИЧ,АРИКОВ ЗАЙРОРИК ЁРУДОВИЧ,Z1CLZH,3
1318,514835,ЗАРИЧЖАНОВ АЧУНДЕР,ЗАРИЧЖАНОВ АЧУНДЕР,DCAYRI,3
695,335672,ШОКИНА ИЧИНА ПАВРОВНА,,4WLL2P,3


Теперь сделаем следующие: посмотрим на все пары приглашатель-отправитель с единым промокодом, возьмём тех, которые встречаются чаще одного раза, и посчитаем сколько вообще каждый приглашатель использовал промокод дважды (и чаще) в отношении одного отправителя, исключим тех, кто встречается до двух раз включительна (поправка на возможных тёзок):

In [26]:
cft_2 = cft.query("признак_скидки == 'По промокоду'") \
    .query("статус_платежа == 'Выдан'") \
    [['id_юзера-приглашателя', 'фио_приглашателя_после_преобразования', 'телефон_отправителя_хэш', 
      'фио_отправителя_после_преобразования', 'id_операции', 'идентификатор_промокода']] \
    .groupby(['id_юзера-приглашателя', 'фио_приглашателя_после_преобразования', 
              'фио_отправителя_после_преобразования', 'идентификатор_промокода'], as_index = False) \
    .agg({"id_операции": "count"}).sort_values(by='id_операции', ascending=False).query("id_операции > 1") \
    .groupby(["id_юзера-приглашателя", "фио_приглашателя_после_преобразования"], as_index = False) \
    .agg({"id_операции": "sum"}) \
    .sort_values(by='id_операции', ascending=False).query("id_операции > 2")

In [27]:
cft_2['id_операции'] = cft_2['id_операции']-1
#вычтем одну "нормальную" операцию

In [28]:
cft_2

,id_юзера-приглашателя,фио_приглашателя_после_преобразования,id_операции
68,233360,РЖУЧАЕВ РАЮКЖОН ЗОИЧШОНОВИЧ,27
135,458182,ЧАШИКФОРА ШОШЧУШ ШАЧИШ,19
41,130381,ШИРИППОВЗРИЙ ЗЕЧБЕЙ ШЧЕРРИЕВИЧ,18
104,370351,АДРУРШАНОНОВ КУЗЗАШО КУЗЗАРИКОВИЧ,17
122,416868,ДОЙДОДОЕВ РИРШОРЖОН КУЧАЗОВИЧ,16
...,...,...,...
71,237325,КУЗОЕВ АФАКРЖОН,2
134,452383,КАШРАКОВ РОКЧОН РАВЧОНОВИЧ,2
66,230944,ЗАРЖИРИНОВ ФАЧОШИРРИН КУШИДУРРОЕВИЧ,2
114,403247,ДАРАН ЕВБЕНИЙ,2


91 щедрый даритель промокода злоупотребил условиями акции.

Вставим ремарку с проверкой уникальности отправителя по самому простому пути: ФИО и ТЕЛЕФОН
(конечно, идентифицировать можно и более сложным путём: можем сравнить получателя платежа, частоту платежей, место назначения и т.п. факторы, указывающие, что платёж всё-таки делал одни отправитель):

In [29]:
evill[['id_юзера-приглашателя', 'фио_приглашателя_после_преобразования', 'телефон_отправителя_хэш', 
      'фио_отправителя_после_преобразования', 'id_операции', 'идентификатор_промокода']].head()

,id_юзера-приглашателя,фио_приглашателя_после_преобразования,телефон_отправителя_хэш,фио_отправителя_после_преобразования,id_операции,идентификатор_промокода
109,330348,ЧУЗЗАКОВ КИЧФОШИР КУКИНОВИЧ,\=2*Xdaaz`>k,ДОЙКУЧОРОВ ДАШЗИЁЧ ЗУРАШЕВИЧ,2707946625,AG5IR4
111,330348,ЧУЗЗАКОВ КИЧФОШИР КУКИНОВИЧ,\=2*Za\bx`8e,ДОЙКУЧОРОВ ДАШЗИЁЧ ЗУРАШЕВИЧ,2707958481,AG5IR4
297,330348,ЧУЗЗАКОВ КИЧФОШИР КУКИНОВИЧ,\=2*Wbaa{g=e,АДРУРРАЕВ АДРУШАРИК АДРУРШАКИЗОВИЧ,2709047649,AG5IR4
433,330348,ЧУЗЗАКОВ КИЧФОШИР КУКИНОВИЧ,\=2*Ze]bui?g,УКАЧОВ КАРЁЧДЕР ЗОИДЖОНОВИЧ,2709771743,AG5IR4
435,330348,ЧУЗЗАКОВ КИЧФОШИР КУКИНОВИЧ,\=2*Wcad{i?f,УКАЧОВ КАРЁЧДЕР ЗОИДЖОНОВИЧ,2709773315,AG5IR4


тут видно, что идентификация производится именно по номеру телефона отправителя, конечно, есть вероятность, что отправители - полные тёзки, но чтобы одним промиком делился отправитель с 18 тёзками - совсем маловероятно

# Поигрались с данными, а теперь можно перейти к поставленным задачам:

0. проанализируем данные и разметить какие операции плохие, а какие нормальные (плохие = клиенты злоупотребили условиями сервиса)

Мысль 0: хорошая операция - единственная в паре приглашатель-отправитель и первая по дате в дублях одной пары;

Мысль 1: по дате история не работает, т.к. в одной паре в одну дату мб несколько переводов "дублей", будем плясать от id и хорошей считать не "первую", а с меньшим порядковым значением id

Сгруппируем по трём параметрам: приглашатель, отправитель платежа и промик; возьмём для каждой строчки самый маленький id (т.е. самый первый):

In [30]:
first_date = cft.groupby(['фио_приглашателя_после_преобразования', 'фио_отправителя_после_преобразования', 
                          'идентификатор_промокода'], as_index = False) \
                .agg({"id_операции": "min"}).rename(columns={"id_операции": "id_1ой_операции"})

Теперь мёрджим наш основной df с полученным по совпадению всё той же тройки: приглашатель, отправитель платежа и промик, т.о. напротив каждой строки получим самый первый id для укаженной комбинации совпадений

In [31]:
cft_first_date = cft.merge(first_date, how='left', on=['фио_приглашателя_после_преобразования', 
                                                       'фио_отправителя_после_преобразования', 
                                                       'идентификатор_промокода'])

Введём колонку 'хорошая_операция', где при совпадении id будем выводить true, а при несовпадении (как раз наш дубль) - false

In [32]:
cft_first_date['хорошая_операция'] = (cft_first_date['id_операции'] == cft_first_date['id_1ой_операции'])

1. сформируем резюме по итогам анализа: кратко предоставим результат анализа (подтвердился ли сигнал, насколько проблема массовая и т.д.)

Отфильтруем переводы со статусом "выдан" и совершённые по промокоду, возьмём те из них, которые мы клеймили плохими:

Почему мы берём переводы со статусом "выдан"? Будем считать, что мы супер лояльная компания, которая анулирует возможность использования промокода только в момент, когда клиент действительно до конца воспользовался услугой. Допустимо использовать иные фильтры при прочих предположениях.

In [33]:
evill_transaction = cft_first_date[['id_операции', 'фио_приглашателя_после_преобразования', 'фио_отправителя_после_преобразования', 
                'идентификатор_промокода', 'дата_отправки', 'хорошая_операция', 'статус_платежа', 'признак_скидки']] \
    .query("статус_платежа == 'Выдан'") \
    .query("признак_скидки == 'По промокоду'") \
    .query("хорошая_операция == False")

In [34]:
cft.query("статус_платежа == 'Выдан'").query("признак_скидки == 'По промокоду'").shape[0]
#1890 всего операций по промокоду

1890

In [35]:
evill_transaction.shape[0]
#Плохих операций

535

In [36]:
535/1890*100
#28% операций по промокоду были совершены со злоупотреблением акции

28.306878306878307

In [37]:
evill_transaction.groupby("идентификатор_промокода", as_index = False).agg({'id_операции': 'count'}) \
                .sort_values("id_операции", ascending=False).shape

(165, 2)

In [38]:
evill_transaction.groupby("идентификатор_промокода", as_index = False).agg({'id_операции': 'count'}) \
                .sort_values("id_операции", ascending=False)['id_операции'].mean()

3.242424242424242

In [39]:
evill_transaction.groupby("идентификатор_промокода", as_index = False).agg({'id_операции': 'count'}) \
                .sort_values("id_операции", ascending=False).query("id_операции > 10")

,идентификатор_промокода,id_операции
51,CIUWW2,23
138,V65XXZ,18
69,FYFWV3,18
26,675LKD,16
97,LSIH97,15
50,CFAV96,14
55,CUFEDH,12
71,G46YBL,12
40,AG5IR4,11
85,IBHIVP,11


In [40]:
evill_transaction.groupby("идентификатор_промокода", as_index = False).agg({'id_операции': 'count'}) \
                .sort_values("id_операции", ascending=False).query("id_операции > 10")['id_операции'].sum()

150

28% операций по промокоду были совершены со злоупотреблением акции (535 штук из 1890

165 промокодов были использованы более одного раза в паре поделившийся/отправитель платежа

в среднем каждым злоупотреблённым промокодом воспользовались 3 раза

10 промокодов было использовано более 10 раз, на них пришлось 150 злоупотреблений

Сигнал подтвердился, проблема действительно хорошо характеризует представленный датасет

На самом деле некрасиво вышло, т.к. значение False также присвоилось и платежам по купону. Конечно, можно пренебреч этим, добавляя фильтр по признаку скидки, а можно заполнин НаН-ами новые столбцы для платежей по купонам, как это сделано с колонкой "идентификатор промокода":

In [41]:
df1 = cft_first_date.query("признак_скидки == 'По промокоду'")
df2 = cft.query("признак_скидки == 'По купону'")
result_cft = pd.concat([df1, df2])

Итоговая таблица с разметкой:

In [43]:
result_cft.head()

,id_операции,дата_отправки,статус_платежа,сумма,валюта,дата_выдачи,направление,фио_отправителя_после_преобразования,телефон_отправителя_хэш,фио_получателя_после_преобразования,...,агент_отправки,город_выдачи,пункт_выдачи,признак_скидки,идентификатор_промокода,id_юзера-приглашателя,фио_приглашателя_после_преобразования,телефон_(приглашатель)_хэш,id_1ой_операции,хорошая_операция
0,2707850631,2018-08-18,Выдан,1000,Российский рубль,2018-08-18,ТАДЖИКИСТАН,АРИКОВ ДАЙФОРУР,\=2$Xh\b}gAk,ЗАНАРУРОВА БУРНОФА ЗОРИДОВНА,...,"""Агент для online переводов""",ДУШАНБЕ,"ST00985 ЗАО ""Казкоммерцбанк Таджикистан""",По промокоду,AIPHXT,348714,РАКАЧФОРА КАНУЧЕШЧИ ШУЧРАЗ,\=2-[ga_wg8m,2.707851e+09,True
1,2708069031,2018-08-18,Выдан,200,Доллар США,2018-08-21,УЗБЕКИСТАН,ЗИРОВОВ ДЕРФОР НАЧФУРРАЕВИЧ,\=2*Ui[`}i9i,PUQMAPOV QAMAT QO'CFQOQOVICF,...,"""Агент для online переводов""",ПАЙАРЫК,"AG60375 ОАКБ ""Агробанк"", Пайарикский филиал",По промокоду,MPUYXI,227215,ЗИРОВОВ ДЕРФОР НАЧФУРРАЕВИЧ,\=2*Tccbza@g,2.708069e+09,True
2,2708071431,2018-08-18,Выдан,200,Доллар США,2018-08-22,УЗБЕКИСТАН,ШОЗАКОВ КАНЗУЧ,\=2$Xe[ezeAi,ЗАНОЕВА НОРИЧА ШАЧИШОВНА,...,"""Агент для online переводов""",ШОФИРКАН,"AH20128 Народный банк, Филиал Шофиркан",По промокоду,LXVH52,56088,ШОЗАКОВ КАНЗУЧ,\=2)Tfbc}`@l,2.708071e+09,True
3,2708306031,2018-08-19,Выдан,300,Доллар США,2018-08-21,УЗБЕКИСТАН,ЗАРУРРОЕВ АФИФДЕР ЧУЗЗАКОВИЧ,\=2-[g]`ya=h,КУЗАЕВ РИРКУЧОР АЗРАКОВИЧ,...,"""Агент для online переводов""",САМАРКАНД,"AG60374 ОАКБ ""Агробанк"", Сиабский филиал",По промокоду,YEM8TE,431901,ЗАРУРРОЕВ АФИФДЕР ЧУЗЗАКОВИЧ,\=2*Vj[auf=k,2.708306e+09,True
5,2709052831,2018-08-20,Выдан,400,Доллар США,2018-08-21,УЗБЕКИСТАН,S'UMOMOV ANDUVAMI ANDUMAMIK O'S'MI,\=2-Xj\gta8e,SASUQOV ANDUMAMIK SUMAMOVICF,...,"""Агент для online переводов""",АНДИЖАН,"AG77527 АКБ ""Узпромстройбанк"", Андиж.Рег.филиа...",По промокоду,2DP7S8,482703,QFAVKATOV NEKZODNEK NAXTIYOQHOP O'S'MI,\=2$Wbagyi8g,2.709053e+09,True


2. Как быстро выявлять подобную активность?

Т.к. у нас промокод принадлежит 1 поделившемуся: выгружать самые частоиспользуемые промокоды сервиса, и там сверять данные отправителя платежа (помимо номера телефона).

Выявлять повторяющиеся сочетания: поделившийся промокодом + отправитель платежа(по ФИО, а не по номеру телефона) + промокод.

Проверять кол-во купонов на одного поделившегося промокодом (наблюдать за пользователями, получившими в среднем больше купонов, чем остальные участники акции).

3. Варианты реагирования

При выявлении нарушений - прекращать действие промокода.

Ввести ограничение на кол-во использований одного промокода.

Или изменить способ идентификации отправителя платежа по промокоду: ранее, полагаю, использовался именно номер телефона, т.к. для одного отправителя платежа он был не уникален. В этом случае можно добавить критериев для идентификации отправител платежа по промокоду.

# 2. Алгоритм проверки персональных данных:

0.  Взаимосвязь ФИО и паспортных данных (серия, номер, дата выдачи): «плохое» изменение характеризуется изменением только первого параметра: замена ФИО, но ПД не изменились (но не наоборот).
1. Взаимосвязь паспортных данных (серия и номер, орган) с датой выдачи паспорта: обновление серии и номера без изменения даты и наоборот – «плохое» изменение.
2. Взаимосвязь гражданства и паспортных данных: «плохое» изменение характеризуется изменением только первого параметра без изменения второго (но не наоборот).
3. Изменение даты рождения, даты выдачи карты – «плохие» изменения, т.к. дата неизменяема.
4. Небольшой временной интервал м/у любым изменением также можно охарактеризовать как «плохое» изменение, если изменение было, например, в течение недели после даты выдачи, вероятно, была допущена ошибка.


Изменения, обратные «плохим» можно считать хорошими. Например:
Сменилась фамилия и паспортные данные (как минимум: серия, номер, дата)

Альтернативный запрос для разметки операций на SQL (в качестве упрощения взял только часть колонок)

In [44]:
first_date = cft.groupby(['фио_приглашателя_после_преобразования', 'фио_отправителя_после_преобразования', 
                          'идентификатор_промокода'], as_index = False) \
                .agg({"id_операции": "min"}).rename(columns={"id_операции": "id_1ой_операции"})

In [45]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
cft.to_sql('cft', con=engine, index=False)

3306

In [46]:
sql_code = """          


select id_операции, 
       cft_2.фио_приглашателя_после_преобразования, 
       cft_2.фио_отправителя_после_преобразования, 
       cft_2.идентификатор_промокода, 
       дата_отправки,
       статус_платежа,
       признак_скидки,
       (id_операции = id_1ой_операции) as хорошая_операция
from 
(
    SELECT *
    FROM cft
    WHERE признак_скидки = 'По промокоду'
) as cft_2
left join 
(
    SELECT фио_приглашателя_после_преобразования,
           фио_отправителя_после_преобразования,
           идентификатор_промокода,
           MIN(id_операции) AS id_1ой_операции
    FROM cft
    WHERE признак_скидки = 'По промокоду'
    GROUP BY фио_приглашателя_после_преобразования,
             фио_отправителя_после_преобразования, 
             идентификатор_промокода
) as cft_3
on cft_2.фио_приглашателя_после_преобразования = cft_3.фио_приглашателя_после_преобразования and
   cft_2.фио_отправителя_после_преобразования = cft_3.фио_отправителя_после_преобразования and
   cft_2.идентификатор_промокода = cft_3.идентификатор_промокода
   
union all

select id_операции, 
       фио_приглашателя_после_преобразования, 
       фио_отправителя_после_преобразования, 
       идентификатор_промокода, 
       дата_отправки,
       статус_платежа,
       признак_скидки,
       NULL as хорошая_операция
from cft
where признак_скидки = 'По купону'


        """
temp = pd.read_sql(sql_code, engine)
temp

,id_операции,фио_приглашателя_после_преобразования,фио_отправителя_после_преобразования,идентификатор_промокода,дата_отправки,статус_платежа,признак_скидки,хорошая_операция
0,2707850631,РАКАЧФОРА КАНУЧЕШЧИ ШУЧРАЗ,АРИКОВ ДАЙФОРУР,AIPHXT,2018-08-18 00:00:00.000000,Выдан,По промокоду,1.0
1,2708069031,ЗИРОВОВ ДЕРФОР НАЧФУРРАЕВИЧ,ЗИРОВОВ ДЕРФОР НАЧФУРРАЕВИЧ,MPUYXI,2018-08-18 00:00:00.000000,Выдан,По промокоду,1.0
2,2708071431,ШОЗАКОВ КАНЗУЧ,ШОЗАКОВ КАНЗУЧ,LXVH52,2018-08-18 00:00:00.000000,Выдан,По промокоду,1.0
3,2708306031,ЗАРУРРОЕВ АФИФДЕР ЧУЗЗАКОВИЧ,ЗАРУРРОЕВ АФИФДЕР ЧУЗЗАКОВИЧ,YEM8TE,2018-08-19 00:00:00.000000,Выдан,По промокоду,1.0
4,2709052831,QFAVKATOV NEKZODNEK NAXTIYOQHOP O'S'MI,S'UMOMOV ANDUVAMI ANDUMAMIK O'S'MI,2DP7S8,2018-08-20 00:00:00.000000,Выдан,По промокоду,1.0
...,...,...,...,...,...,...,...,...
3301,2723522069,,EM KOPQTAPTIP QUDOMSOVICF,None,2018-09-17 00:00:00.000000,Готов к выдаче,По купону,NaN
3302,2723557519,,АДРУАРИЕВ АРИЖОН,None,2018-09-17 00:00:00.000000,Выдан,По купону,NaN
3303,2723574577,,ЗАИРОВ АДРУРШАЗОШ ЗОИЧОВИЧ,None,2018-09-17 00:00:00.000000,Выдан,По купону,NaN
3304,2723596319,,ЗОРИШОВ КИЧФОДЕР КИЧФОЕВИЧ,None,2018-09-17 00:00:00.000000,Готов к выдаче,По купону,NaN
